Adressa Dataset: Gulla, J. A., Zhang, L., Liu, P., Özgöbek, Ö., & Su, X. (2017, August). The Adressa dataset for news recommendation. In Proceedings of the International Conference on Web Intelligence (pp. 1042-1048). ACM.

In [1]:
import pandas as pd
from tqdm import tqdm
import os
import json
from utils.adressa_util import preprocessing
from pathlib import Path
#from utils.adressa_util.entity_linking import add_entities


In [2]:
one_week_path = './adressa/one_week'
content_news = './adressa/content_refine'
out_path = './adressa/mind_format'

In [3]:
# Getting hash of each news and mapping it to a unique id like MIND N***
hash_title, hash2id = preprocessing.news_title(one_week_path,content_news )

1513739it [00:23, 64920.22it/s]
1356987it [00:18, 72866.58it/s] 
1087223it [00:19, 56347.76it/s]
1327429it [00:24, 54956.28it/s]
1613128it [00:27, 59019.86it/s]
1648346it [00:28, 58653.11it/s]
1496417it [00:30, 48405.23it/s]


In [4]:
out_path = Path(out_path)
preprocessing.write_news_files(hash_title,hash2id,out_path)

100%|██████████| 17692/17692 [00:00<00:00, 1007585.19it/s]


In [5]:
article_hash_content = {}
for filename in os.listdir(content_news):
    with open(os.path.join(content_news, filename), "r") as f:
        article = json.load(f)
        article_hash = article['id']
        article_dict = {d['field'] : d['value'] for d in article['fields']}
        article_hash_content[article_hash] = article_dict

In [6]:
from utils.adressa_util.preprocessing import write_news_files_full
write_news_files_full(hash_title,hash2id,out_path,article_hash_content)

100%|██████████| 17692/17692 [00:00<00:00, 372243.51it/s]


In [7]:
with open('adressa/mind_format/train/news.tsv', 'r') as fp:
    count= 1
    for line in fp:
        fields = line.split('\t')
        assert int(fields[0].strip('N')) == count
        count += 1

In [7]:
## DA eliminare

from utils.adressa_util.preprocessing import entities_news
out_path = './adressa/mind_format/train/news_new.tsv'
out_path = Path(out_path)
entities_list = entities_news(out_path)

In [8]:
# Aggiungere parte attributi

In [9]:
## FINE DA ELIMINARe

# Do not execute or it will download entire refined pretrained model 30+GB
out_path = './adressa/mind_format'
out_path = Path(out_path)
add_entities(out_path)

In [21]:
from utils.adressa_util.preprocessing import process_users
uid2index, user_info = process_users(one_week_path, hash2id, content_news)

1513739it [00:27, 54950.67it/s]
1356987it [00:20, 65248.39it/s] 
1087223it [00:19, 56441.51it/s]
1327429it [00:26, 50709.23it/s]
1613128it [00:30, 53019.52it/s]
1648346it [00:30, 53267.90it/s]
1496417it [00:33, 44908.82it/s]


In [23]:
for uid in tqdm(user_info):
    user_info[uid].sort_click()

100%|██████████| 601215/601215 [00:09<00:00, 64349.95it/s]


In [34]:
from utils.adressa_util.preprocessing import construct_behaviors

train_lines = []
test_lines = []

# Randomly sample NEG_NUM negative impression for every positive behavior
NEG_NUM = 20
for uindex in tqdm(user_info):
    uinfo = user_info[uindex]
    click_news = uinfo.click_news
    train_news = uinfo.train_news
    test_news = uinfo.test_news
    train_user,test_user = construct_behaviors(hash_title, uindex, click_news, train_news, test_news, NEG_NUM)
    train_lines.extend(train_user)
    test_lines.extend(test_user)

100%|██████████| 601215/601215 [02:24<00:00, 4165.08it/s] 


In [35]:
train_lines

['null\t0\tnull\tN1 N1794 N2082 N3549 N4194 N4479 N4197 N4769 N4657 N6280 N4657 N4367 N4880 N5117 N4185 N4147 N4352 N4318 N4176 N4135 N4079\tN5354-1 N8034-0 N13077-0 N9946-0 N7418-0 N12976-0 N851-0 N9878-0 N1618-0 N9094-0 N8732-0 N476-0 N5485-0 N17517-0 N15571-0 N8079-0 N17245-0 N13836-0 N6407-0 N2688-0 N6541-0\n',
 'null\t1\tnull\tN2 N2082 N1960\tN7393-1 N15434-0 N12274-0 N8527-0 N9325-0 N9846-0 N9883-0 N4920-0 N10766-0 N8458-0 N633-0 N15179-0 N9345-0 N8865-0 N1770-0 N8825-0 N8180-0 N13822-0 N9588-0 N9428-0 N7550-0\n',
 'null\t5\tnull\tN1 N3 N351 N1 N2082 N13 N4731 N8319 N4197 N6560 N4199 N4627 N5940 N978 N4367 N4119 N4092 N6797 N4899 N6797 N1991 N5716\tN5004-1 N6286-0 N16072-0 N2150-0 N12852-0 N7839-0 N3431-0 N13477-0 N6116-0 N3556-0 N14685-0 N3792-0 N14359-0 N5053-0 N10547-0 N2079-0 N12287-0 N6967-0 N963-0 N8861-0 N449-0\n',
 'null\t6\tnull\tN1 N2437 N2082 N1960 N3312 N4197 N4367 N4479 N4367 N5643 N4197 N4119 N4181 N4185 N5364 N4318 N4111 N4899 N4231 N4136 N4133\tN4073-1 N8745-0 N10

In [36]:
from sklearn.model_selection import train_test_split

test_split_lines, valid_split_lines = train_test_split(
    test_lines, test_size=0.2, random_state=2023
)
with open(out_path / "train" / "behaviors.tsv", "w", encoding="utf-8") as f:
    f.writelines(train_lines)

with open(out_path / "valid" / "behaviors.tsv", "w", encoding="utf-8") as f:
    f.writelines(valid_split_lines)

with open(out_path / "test" / "behaviors.tsv", "w", encoding="utf-8") as f:
    f.writelines(test_split_lines)

In [ ]:
## Adding Entities to NO news

In [ ]:
#!pip install https://github.com/amazon-science/ReFinED/archive/refs/tags/V1.zip --quiet
#!pip install ctranslate2 --quiet
#!ct2-transformers-converter --model alirezamsh/small100 --output_dir no_en_model

In [78]:
assert os.path.exists('no_en_model'), 'Please run cell above to install the translation model'
from utils.adressa_util.entity_linking import add_entities
path_news_with_entities = add_entities(Path('./adressa/mind_format'))

AssertionError: Please Run Cell above to install the translation model

In [80]:
def entities_news(path_news):
    entities = set()
    # Read entities from train news
    with open(path_news) as fp:
        for line in fp:
            newsid, vert, subvert, title, abstract, url, entity_info_title, entity_info_abstract = line.strip().split(
                '\t')
            # Add entities in the title
            for entity in eval(entity_info_title):
                entities.add(entity["WikidataId"])
            # Add entities in the abstract
            for entity in eval(entity_info_abstract):
                entities.add(entity["WikidataId"])
    return entities

entities_new = entities_news('adressa/mind_format/train/news_new.tsv')
entities_new
# Run a check to see that not all new entities are encoded

{'Q919191',
 'Q91683231',
 'Q28898',
 'Q1070749',
 'Q84144245',
 'Q11967663',
 'Q948900',
 'Q1773159',
 'Q259745',
 'Q17107347',
 'Q687308',
 'Q12321978',
 'Q204365',
 'Q648786',
 'Q331483',
 'Q11977358',
 'Q11989782',
 'Q628',
 'Q776909',
 'Q2449414',
 'Q11972186',
 'Q2815524',
 'Q11686878',
 'Q4919607',
 'Q2208855',
 'Q150939',
 'Q17711742',
 'Q1364914',
 'Q543556',
 'Q9531',
 'Q25430776',
 'Q315773',
 'Q101122307',
 'Q37108974',
 'Q11967949',
 'Q3028607',
 'Q19974483',
 'Q23322314',
 'Q3485383',
 'Q12007812',
 'Q15725641',
 'Q806322',
 'Q256738',
 'Q7787084',
 'Q485116',
 'Q3357643',
 'Q43510',
 'Q287690',
 'Q4736686',
 'Q12007158',
 'Q11998138',
 'Q570651',
 'Q3895',
 'Q3346699',
 'Q1479',
 'Q1800417',
 'Q219586',
 'Q16898977',
 'Q18243208',
 'Q1355628',
 'Q21086634',
 'Q8918',
 'Q209782',
 'Q14751',
 'Q3642293',
 'Q7061033',
 'Q3870',
 'Q4586138',
 'Q503763',
 'Q154912',
 'Q499450',
 'Q1493165',
 'Q252749',
 'Q17776485',
 'Q849046',
 'Q9500',
 'Q5388750',
 'Q97684861',
 'Q18822',


In [83]:
# Saving wiki id, encoded using same pipeline as jobs
pd.DataFrame(entities_new, columns=['entity']).to_csv('adressa/mind_format/train/wikidata_ids.txt', index=False, header=False)
# This part was implemented in notebook, TODO: Implement relation2vec
#from utils.adressa_util.entity2vec import entities2vec
#entities2vec(entities_new, out_path / "train" / "entity_embedding_.tsv")

In [29]:
# #A Append entities to NO news
# id_entities_ = {}
# with open('adressa/mind_format/train/news_new.tsv', 'r') as eng_entities:
#     for line in eng_entities:
#         fields = line.split('\t')
#         id_news = fields[0]
#         entities_title = fields[-2]
#         entities_abs = fields[-1]
#         id_entities_[id_news] = [entities_title, entities_abs]
# nor_news = []
# with open('adressa/mind_format/train/news.tsv', 'r') as old_entities:
#     for line in old_entities:
#         fields = line.split('\t')
#         id_news = fields[0]
#         fields[-2] = id_entities_[id_news][0]
#         fields[-1] = id_entities_[id_news][1]
#         nor_news.append(fields)
# with open('adressa/mind_format/train/news_no_eng_entities.tsv', "w") as new_file:
#     for news in nor_news:
#         new_file.write(("\t".join(news)))

In [27]:
input_file = 'adressa/mind_format/test/behaviors.tsv'
output_file = 'adressa/mind_format/test/behaviors_.tsv'
with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        lines = f_in.readlines()
        for line in lines:
            line = line.strip('\n').split('\t')
            news_0 = line[3].split(' ')
            news_0 = ' '.join(['N'+x for x in news_0])
            if news_0 == 'N':
                news_0 = ''
            news_1 = line[4].split(' ')
            news_1 = ' '.join(['N'+x for x in news_1])
            line[3] = news_0
            line[4] = news_1
            f_out.write('\t'.join(line)+'\n')

In [25]:
line

['null',
 '235869',
 'null',
 '',
 'N8802-1 N8613-0 N7174-0 N6213-0 N3803-0 N14586-0 N15130-0 N2213-0 N13034-0 N16287-0 N13287-0 N2938-0 N13701-0 N6432-0 N12349-0 N7263-0 N5114-0 N3880-0 N16672-0 N11559-0 N8287-0']